<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#프로젝트-:-뉴스-기사-클러스터,-시각화" data-toc-modified-id="프로젝트-:-뉴스-기사-클러스터,-시각화-1">프로젝트 : 뉴스 기사 클러스터, 시각화</a></span><ul class="toc-item"><li><span><a href="#과제" data-toc-modified-id="과제-1.1">과제</a></span></li><li><span><a href="#풀이" data-toc-modified-id="풀이-1.2">풀이</a></span><ul class="toc-item"><li><span><a href="#뉴스-기사-읽어오기,-전처리" data-toc-modified-id="뉴스-기사-읽어오기,-전처리-1.2.1">뉴스 기사 읽어오기, 전처리</a></span></li><li><span><a href="#EM" data-toc-modified-id="EM-1.2.2">EM</a></span></li><li><span><a href="#DTM,-TDM,-TWM" data-toc-modified-id="DTM,-TDM,-TWM-1.2.3">DTM, TDM, TWM</a></span></li><li><span><a href="#어휘-클러스터" data-toc-modified-id="어휘-클러스터-1.2.4">어휘 클러스터</a></span></li><li><span><a href="#시각화" data-toc-modified-id="시각화-1.2.5">시각화</a></span></li></ul></li></ul></li></ul></div>

## 프로젝트 : 뉴스 기사 클러스터, 시각화

### 과제

- 뉴스 데이터에 적용하기 (iter=10)


- WordCloud 만들기(명사만) => SSE 확인


- K=2가 아닌, < 10까지 변화 시키기

### 풀이

#### 뉴스 기사 읽어오기, 전처리

In [53]:
import os
from collections import defaultdict
from math import log
from math import exp

In [54]:
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [55]:
from functions.naver import NewsScraping

from functions.info_retrieval import max_tf, raw_idf
from functions.info_retrieval import smoothig_idf
from functions.info_retrieval import clean_collection
from functions.info_retrieval import extend_lexicon
from functions.info_retrieval import inverted_index_with_tf
from functions.info_retrieval import get_tdm_from_dtm
from functions.info_retrieval import tdm2twm
from functions.info_retrieval import evaluate_idf

In [56]:
category = {
    '정치': '00',
    '경제': '01',
    '사회': '02',
    '생활문화': '03',
    '세계': '04',
    'IT과학': '05'
}

inverse_category = {
    '00': '정치',
    '01': '경제',
    '02': '사회',
    '03': '생활문화',
    '04': '세계',
    '05': 'IT과학'
}

In [57]:
news = NewsScraping()

In [58]:
filenames = news.get_filenames(all_folder=True)
len(filenames)

540

In [59]:
collection = []   # tuple(filename, content)들의 list

for filename in filenames:
    collection.append((filename.split("/")[-1], news.get_content(filename)))
    
len(collection)

540

In [60]:
cleaned_collection = clean_collection(collection)

In [61]:
extended_collection = []
corpus_count = len(cleaned_collection)

for idx, (filename, corpus) in enumerate(cleaned_collection):
    extended_lexicon = extend_lexicon(corpus)
    extended_collection.append((filename, extended_lexicon))
    if (idx + 1) % 30 == 0:
        print(corpus_count, "개 documents 중", idx + 1, "개 documents의 lexicon 추출 완료")

print(idx + 1, "개 documents의 lexicon 추출 완료")

540 개 documents 중 30 개 documents의 lexicon 추출 완료
540 개 documents 중 60 개 documents의 lexicon 추출 완료
540 개 documents 중 90 개 documents의 lexicon 추출 완료
540 개 documents 중 120 개 documents의 lexicon 추출 완료
540 개 documents 중 150 개 documents의 lexicon 추출 완료
540 개 documents 중 180 개 documents의 lexicon 추출 완료
540 개 documents 중 210 개 documents의 lexicon 추출 완료
540 개 documents 중 240 개 documents의 lexicon 추출 완료
540 개 documents 중 270 개 documents의 lexicon 추출 완료
540 개 documents 중 300 개 documents의 lexicon 추출 완료
540 개 documents 중 330 개 documents의 lexicon 추출 완료
540 개 documents 중 360 개 documents의 lexicon 추출 완료
540 개 documents 중 390 개 documents의 lexicon 추출 완료
540 개 documents 중 420 개 documents의 lexicon 추출 완료
540 개 documents 중 450 개 documents의 lexicon 추출 완료
540 개 documents 중 480 개 documents의 lexicon 추출 완료
540 개 documents 중 510 개 documents의 lexicon 추출 완료
540 개 documents 중 540 개 documents의 lexicon 추출 완료
540 개 documents의 lexicon 추출 완료


#### EM

In [62]:
# Similarity = Euclidean Distance
from math import sqrt

def euclidean(x1, x2):
    _sum = 0.0
    for i in range(len(x1)):
        _sum += (x1[i] - x2[i]) ** 2
        
    return sqrt(_sum)

In [63]:
def cosine(x1, x2):
    _sum = 0.0
    x1_length = 0.0
    x2_length = 0.0
    
    for i in range(len(x1)):
        _sum += x1[i]*x2[i]
    
    return _sum / (euclidean(x1, [0 for _ in range(len(x1))]) * euclidean(x2, [0 for _ in range(len(x1))]))

In [64]:
# 2개 이상을 처리하는 expectation
def expectation(data, clusters, k=3, cos=True):
    distance = list()
    metric = cosine if cos else euclidean
    rvalue = max if cos else min
    
    distance = list()

    for i in range(k):
        distance.append(metric(data, clusters[i]))
        
    return distance.index(rvalue(distance))

In [65]:
# 두개 이상을 처리하는 maximization
def maximization(data):
    N = len(data)
    _sum = list(0 for _ in range(len(data[0])))
    
    for _ in data:
        for i in range(len(_)):
            _sum[i] += _[i]

    return tuple(_/N for _ in _sum)

In [66]:
def sse(data, centroid):   # sum squared error
    _sum = 0.0
    
    for _ in data:
        _sum += euclidean(_, centroid)
        
    return _sum

In [67]:
# # Randomly K=3, N=100, Centroids=Clusters => rnk
# iterCount = 6
# sseList = list()

# cluster = sample(data, K)    # 실행 할 때 마다 초기화 필요

# for _ in range(iterCount):
#     rnk = list(list(0 for _ in range(K)) for _ in range(N))

#     for i in range(N):
#         k = expectation(data[i], cluster)
#         rnk[i][k] = 1

#     _sum = 0.0
#     oldCluster = list()
    
#     for i in range(K):
#         dataset = [data[j] for j in range(N) if rnk[j][i]]
#         _sum += sse(dataset, cluster[i])
#         oldCluster.append(cluster[i])
#         cluster[i] = maximization(dataset)
        
#     print("Iteration: {0} / SSE:{1}".format(_+1, _sum))

#     for i in range(N):
#         plt.scatter(data[i][0], data[i][1], alpha=0.3, color=colormap[rnk[i].index(max(rnk[i]))])

#     for i in range(K):
#         plt.plot((oldCluster[i][0], cluster[i][0]), (oldCluster[i][1], cluster[i][1]), "r-")
#         plt.scatter(cluster[i][0], cluster[i][1], color=colormap[i], linewidths=4, edgecolors=colormap[-1])
    
#     plt.show()
    
#     sseList.append(_sum)

In [68]:
# # Randomly K=3, N=100, Centroids=Clusters => rnk
# iterCount = 6
# sseList = list()

# # https://matplotlib.org/examples/color/named_colors.html
# colormap = ("r", "g", "b", "c", "m", "y", "gray", "gold", "navy", "ivory", "violet", "pink", "brown", "k")

# for k in range(2, 10):
#     cluster = sample(data, k)    # 실행 할 때 마다 초기화 필요

#     for _ in range(iterCount):
#         rnk = list(list(0 for _ in range(k)) for _ in range(N))

#         for i in range(N):
#             j = expectation(data[i], cluster, k=k, cos=True)
#             rnk[i][j] = 1

#         _sum = 0.0
#         oldCluster = list()

#         for i in range(k):
#             dataset = [data[j] for j in range(N) if rnk[j][i]]
#             _sum += sse(dataset, cluster[i])
#             oldCluster.append(cluster[i])
#             cluster[i] = maximization(dataset)

#     sseList.append(_sum)
        
#     print("Iteration: {0} / SSE:{1}".format(k, _sum))

#     for i in range(N):
#         plt.scatter(data[i][0], data[i][1], alpha=0.3, color=colormap[rnk[i].index(max(rnk[i]))])

#     for i in range(k):
#         plt.plot((oldCluster[i][0], cluster[i][0]), (oldCluster[i][1], cluster[i][1]), "r-")
#         plt.scatter(cluster[i][0], cluster[i][1], color=colormap[i], linewidths=4, edgecolors=colormap[-1])

#     plt.show()

# plt.plot(range(2, 10), sseList, "r-")
# plt.show()

#### DTM, TDM, TWM

In [69]:
global_lexicon, global_posting, global_document, dtm = inverted_index_with_tf(extended_collection)
len(global_lexicon), len(global_posting), len(global_document), len(dtm)

(92098, 379911, 540, 540)

In [70]:
tdm = get_tdm_from_dtm(dtm)
twm, dtw = tdm2twm(tdm, global_document)
global_lexicon_idf, global_document_weight = evaluate_idf(global_lexicon, global_posting, global_document)

In [71]:
# 각각의 문서를 길이가 동일한 다차원 벡터로 만들기 (크기는 전체 Vocabulary 크기)
N = len(dtm)   # 전체 document 수
V = len(tdm)   # term의 전체 차원 수(vocabulary 수)

cluster = list()
docVector = list(list(0.0 for _ in range(V)) for _ in range(N))

for i in range(N):
    for j, t in enumerate(tdm.keys()):
        docVector[i][j] = twm[t][i]

In [72]:
# len(docVector)
# print(docVector)

#### 어휘 클러스터

In [73]:
from random import sample

K = 3

cluster = sample(docVector, K)
len(cluster), len(docVector[0])

(3, 92098)

In [74]:
sseList = list()

for _ in range(10):
    rnk = list(list(0.0 for _ in range(K)) for _ in range(N))

    for i, d in enumerate(docVector):
        c = expectation(d, clusters=cluster, k=K, cos=False)
        rnk[i][c] = 1
        
    _sum = 0.0
    
    for j, c in enumerate(cluster):
        dataset = [docVector[i] for i in range(N) if rnk[i][j]]
        _sum += sse(dataset, c)
        cluster[j] = maximization(dataset)
        
    sseList.append(_sum)

IndexError: list index out of range

In [ ]:
len(sseList)

In [ ]:
plt.plot(range(1, 11), sseList, "r-")

#### 시각화

In [ ]:
clusterDataset = list()

for j, c in enumerate(cluster):
    dataset = [i for i in range(N) if rnk[i][j]]
    clusterDataset.append(dataset)

In [ ]:
voca = list(twm.keys())
topic = list()

for c in cluster:
    wordList = defaultdict(float)
    
    for i, w in enumerate(c):
        wordList[voca[i]] = w
        
    topic.append(dict(sorted(wordList.items(), key=lambda x: x[1], reverse=True)[:5]))    # {word, value}
    print(sorted(wordList, key=wordList.get, reverse=True)[:5])      # [word]

In [ ]:
from wordcloud import  WordCloud
fontPath = "Libray/Fonts/AppleGothic.ttf"
wc = WordCloud(font_path=fontPath, background_color="white")
wc.generate_from_frequencies(topic[0])
wc.to_image()